To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# 💳 Transaction Enrichment

**Hook:** *"Turn cryptic merchant codes into actionable business intelligence"*

Raw transaction data is a mess - "AMZ*DIGITAL*7" or "SQ*COFFEE SHOP". Traditional categorization uses rigid rules and misses context. Watch AI enrich transactions with semantic understanding, turning payment data into business insights.

**What you'll see in this 2-minute demo:**
- 💳 **Cryptic transaction data** - Real-world payment descriptions and merchant codes
- 🎯 **Smart categorization** - AI understands context beyond keyword matching
- 📊 **Business categories** - Structured expense tracking and spending analysis
- 💼 **Expense insights** - Actionable data for financial management

This transforms financial chaos into structured business intelligence.

In [ ]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import Literal

# ⚡ Configure for financial data processing
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="transaction_enrichment_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "fintech": fc.OpenAILanguageModel(model_name="gpt-4o-mini", rpm=500, tpm=200_000),
            # "fintech": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "fintech": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=500, tpm=200_000)
        }
    )
))

print("✅ Financial transaction enrichment configured")

## 💳 Step 1: Transaction Enrichment Schema

Define the structured business data we want to extract from raw transactions:

In [ ]:
# 💼 Transaction intelligence schema using Pydantic
class TransactionEnrichment(BaseModel):
    business_category: Literal["food", "software", "travel", "office", "marketing", "utilities", "other"] = Field(description="Standardized business category")
    vendor_name: str = Field(description="Clean, readable business name")
    expense_type: Literal["recurring", "one-time", "subscription"] = Field(description="Spending pattern classification")
    tax_deductible: bool = Field(description="Whether expense is likely tax deductible for business")

print("💼 Transaction Enrichment Schema:")
print("   • business_category: Standardized expense category")
print("   • vendor_name: Human-readable business name")
print("   • expense_type: Spending pattern classification")
print("   • tax_deductible: Business tax deduction eligibility")

## 💸 Step 2: Cryptic Transaction Data

Real-world transaction descriptions that are hard to interpret:

In [ ]:
# 💸 Real transaction data with cryptic descriptions
transactions = session.create_dataframe([
    {"id": "TXN001", "description": "AMZ*DIGITAL*AWS*7G2H4", "amount": -89.45, "date": "2024-01-15"},
    {"id": "TXN002", "description": "SQ*BLUE BOTTLE COFFEE", "amount": -12.50, "date": "2024-01-15"},
    {"id": "TXN003", "description": "GITHUB.COM*ENTERPRISE", "amount": -420.00, "date": "2024-01-01"},
    {"id": "TXN004", "description": "UBER*EATS*SF*HELP.UBER", "amount": -35.80, "date": "2024-01-14"},
    {"id": "TXN005", "description": "ZOOM.US*ZOOM*US", "amount": -14.99, "date": "2024-01-01"},
    {"id": "TXN006", "description": "OFFICE DEPOT #2847", "amount": -67.34, "date": "2024-01-12"},
    {"id": "TXN007", "description": "GOOGLE*ADS*123ABC", "amount": -250.00, "date": "2024-01-10"}
])

print("💸 Raw Transaction Data - Notice the cryptic descriptions:")
transactions.show()

## 🧠 Step 3: AI Transaction Enrichment

Use AI to decode transaction descriptions and extract business intelligence:

In [ ]:
# 🧠 AI-powered transaction enrichment
enriched_transactions = transactions.select(
    "id",
    "description",
    "amount",
    "date",
    fc.semantic.extract(
        "description",
        TransactionEnrichment,
        model_alias="fintech"
    ).alias("enrichment")
).cache()  # Cache enrichment results

# Extract structured data for analysis
transaction_insights = enriched_transactions.select(
    "id",
    "description",
    "amount",
    enriched_transactions.enrichment.vendor_name.alias("vendor"),
    enriched_transactions.enrichment.business_category.alias("category"),
    enriched_transactions.enrichment.tax_deductible.alias("deductible"),
    enriched_transactions.enrichment.expense_type.alias("type")
)

print("🧠 ENRICHED TRANSACTION DATA:")
transaction_insights.show()

## 📊 Step 4: Business Intelligence Dashboard

Generate actionable insights for expense management and tax planning:

In [ ]:
# 📊 Business intelligence analysis
total_transactions = transactions.count()
deductible_count = transaction_insights.filter(fc.col("deductible")).count()
deductible_percentage = (deductible_count / total_transactions) * 100

print("📊 FINANCIAL INTELLIGENCE DASHBOARD:")
print(f"   • Total transactions processed: {total_transactions}")
print(f"   • Tax-deductible transactions: {deductible_count} ({deductible_percentage:.0f}%)")

# Category breakdown
category_analysis = transaction_insights.group_by("category").agg(
    fc.count("*").alias("transaction_count")
).order_by(fc.desc("transaction_count"))

print("\n💼 SPENDING BY CATEGORY:")
category_analysis.show()

print("\n🎯 BUSINESS BENEFITS:")
print("   • Automated expense categorization saves hours of manual work")
print("   • Tax deductible identification maximizes business deductions")
print("   • Vendor normalization enables accurate spending analysis")
print("   • Subscription detection helps manage recurring costs")

In [ ]:
session.stop()